In [12]:
import gmaps
import gmaps.geojson_geometries
import gmaps.datasets
gmaps.geojson_geometries.list_geometries()
counties_geojson = gmaps.geojson_geometries.load_geometry("us-counties")

In [11]:
counties_ls = counties_geojson['features']
fig = gmaps.figure()
county_layer = gmaps.geojson_layer(counties_geojson)
fig.add_layer(county_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [182]:
import random
import pandas as pd

fipps_ls = []
c_percent_ls = []
s_percent_ls = []
f_percent_ls = []

for county in counties_ls:
    props = county['properties']
    county_state = props["STATE"]
    county_num = props["COUNTY"]
    county_fipps = county_state+county_num
    fipps_ls.append(county_fipps)
    c = random.uniform(0.0,0.5)*100
    s = random.uniform(0.0,0.5)*100
    c_percent_ls.append(c)
    s_percent_ls.append(s)
    f_percent_ls.append(100-c-s)

df = pd.DataFrame({'FIPPs':fipps_ls,'pct_chinese':c_percent_ls,'pct_spanish':s_percent_ls,"pct_french":f_percent_ls})
df.set_index("FIPPs")

In [190]:
from matplotlib.cm import viridis
from matplotlib.colors import to_hex
colors = []

def shade_pct(df, language):
    counties = []
    for county in df.iterrows():
        counties.append(county)
    #return counties
        county_fipps = county[1]["FIPPs"]
        pct = county[1][F"pct_{language}"]
        #print(pct)
        norm_pct = pct/100
        mpl_color = viridis(norm_pct)
        gmaps_color = to_hex(mpl_color, keep_alpha=False)
        colors.append(gmaps_color)
    return counties, colors

d, c = shade_pct(df, "chinese")
#[0][1]["FIPPs"]

In [191]:
state_geojson = gmaps.geojson_geometries.load_geometry("us-states")

In [196]:
fig = gmaps.figure()
pct_layer = gmaps.geojson_layer(counties_geojson, stroke_weight = 0.5,
                                fill_color=c, fill_opacity=0.8)
state_layer = gmaps.geojson_layer(state_geojson, stroke_weight = 2,
                                  fill_opacity=0)
fig.add_layer(pct_layer)
fig.add_layer(state_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [186]:
l_colors = {"spanish":"#FF8E00","french":"#006FFF","chinese":"#FF0000"}
languages = list(l_colors.keys())
#l_colors = {"chinese":"#FF0000"}
#languages = list(l_colors.keys())

def shade_lan(df):
    colors = []
    counties = []
    for county in df.iterrows():
        gmaps_color = "#FFFFFF"
        county_fipps = county[1]["FIPPs"]
        plur_lan = 0
        #plur_lan = 50
        for lan in languages: 
            pct = county[1][F"pct_{lan}"]
            if pct > plur_lan:
                plur_lan = pct
                gmaps_color = l_colors[lan]
        colors.append(gmaps_color)
    return colors
colors = shade_lan(df)

In [187]:
fig = gmaps.figure()
pct_layer = gmaps.geojson_layer(counties_geojson, fill_color=colors)
fig.add_layer(pct_layer)
fig

Figure(layout=FigureLayout(height='420px'))